In [27]:
# Import dependencies and required magics
# Importing TextBlob
from textblob import TextBlob
import requests
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd 
import re
import plotly.express as px
import text2emotion as te

# DOWNLOAD NLTK USING THIS SCRIPT HERE: nltk.download()
# nltk.download()
import nltk
import spacy

import datetime
#Learn more about SpaCY: https://spacy.io/

# RUN THE 3 SCRIPTS BELOW IN YOUR COMMAND PROMPT TO USE SPACY FOR TEXT ANALYSIS
# 1) pip install -U spacy
# 2) python -m spacy download en_core_web_sm
# 3) python -m spacy download en
nlp = spacy.load('en_core_web_sm')

In [28]:
df = pd.read_csv("File Path", encoding="unicode_escape")

df

,ï»¿Posted Date,Review Id,Review Text
0,5/30/21,gp:AOqpTOFUTH9AaCE6WSwdry4cgHPaFVo1OHEt6aXJ7W8...,"""Excellent app good *****""love it"
1,7/6/21,gp:AOqpTOG7SH1RbnY5gf5y3vFxxBjTyreulWiFZC9bHe1...,#Renukakhatri70#Nrpali_tiktok#more views.
2,6/24/21,gp:AOqpTOGZY2ixXJl6AD7zMRyzDibMfxmGtpDnLI8i-bh...,#for you page for u page #viral
3,8/19/21,gp:AOqpTOF9C9qyb_BgWvUZFBSkm126pi6bEqG7n-7L5ab...,#i love this app so mush i will watch it all d...
4,2/15/21,gp:AOqpTOGu6vG6HpQNLql9403K1hjVw15A802GwE1cL5a...,(pov) your bored amd reading the reviews on th...
...,...,...,...
687504,3/1/21,gp:AOqpTOE52fQlIsL_N6C-VGqyeqpNtcmc3Mm9Dn9ajDz...,Shafqat jokhio
687505,2/17/21,gp:AOqpTOHIxBeUZbjFbawQ72jR-AiN8G3QkG59nXp23_q...,Shafqatalikhan
687506,2/13/21,gp:AOqpTOEaC9aGmSHNFq_Y9id2yif-6rl3tthLzgjP9E_...,Shah g
687507,3/5/21,gp:AOqpTOHVQ0kNO0Ru3oZzjPNRQuBBeRKixp-1uuFCxbe...,Shahad jutt g


In [29]:
# Grabs the first 200 reviews from each month in 2021

df_Jan = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 1]
df_Feb = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 2]
df_Mar = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 3]
df_Apr = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 4]
df_May = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 5]
df_June = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 6]
df_Jul = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 7]
df_Aug = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 8]
df_Sept = df[pd.to_datetime(df['ï»¿Posted Date']).dt.month == 9]

In [30]:
# Creates new dataframes for each month with first 200 records from previous step

df1_Jan = df_Jan.head(200)
df1_Feb = df_Feb.head(200)
df1_Mar = df_Mar.head(200)
df1_Apr = df_Apr.head(200)
df1_May = df_May.head(200)
df1_June = df_June.head(200)
df1_Jul = df_Jul.head(200)
df1_Aug = df_Aug.head(200)
df1_Sept = df_Sept.head(200)

In [33]:
# Places the dataframes into an array to be combined in the next step

sample_frames = [df1_Jan, df1_Feb, df1_Mar, df1_Apr, df1_May, df1_June, df1_Jul, df1_Aug, df1_Sept]

In [36]:
# Combines the new dataframes from the above step for analysis

result = pd.concat(sample_frames)
result

,ï»¿Posted Date,Review Id,Review Text
14,1/25/21,gp:AOqpTOFQicuONNxgRaALhaZ9UA1s6k6keMQNTOU0jpK...,1/24/21 having issues with the app
30,1/26/21,gp:AOqpTOEVlTrH8M0CTZC4Ce-14LLz3wK6IBrYJV84tnF...,5 stars for this cool up i really like ot beco...
50,1/2/21,gp:AOqpTOGfOzVPTGC1kMiYQ8CvelsxFxhjT37PZFOn51D...,A good application for entertainment
59,1/29/21,gp:AOqpTOFb3ce2iH4C1ziXbhXs0BtrtKvZaCiGL-t55Xy...,A little glitchy
66,1/16/21,gp:AOqpTOEVHOgTH3VIR5m-HjcnJoSxM8XxhbQN6uHr8L-...,A very addicting app i love it its fun
...,...,...,...
3036,9/14/21,gp:AOqpTOHMJvJiILS3s6KprRo3MvYgTgTNXjq6TtuTMet...,Awesame app but Pakistan his ban
3037,9/14/21,gp:AOqpTOHc5wHpPiDwwdMPqhgFDDbI7eed5yXpzrGHr_M...,Awesom love it
3040,9/11/21,gp:AOqpTOHZTxbM_ZZ1C_esfhT6lGfwhyIIrXdThS5bffL...,Awesome App i like u
3048,9/4/21,gp:AOqpTOG-v_HOHpC4JzycMOiye-y9dDbWjJaD4q6ZkHJ...,Awesome apps bro


In [9]:
# Checks data types of columns
df.dtypes

ï»¿Posted Date    object
Review Id         object
Review Text       object
dtype: object

In [37]:
# Adds polarity column | Credits to Stack Overflow
def polarity_calc(text):
    try:
        return TextBlob(text).polarity
    except:
        return None

result['polarity'] = result['Review Text'].apply(polarity_calc)

In [38]:
result

,ï»¿Posted Date,Review Id,Review Text,polarity
14,1/25/21,gp:AOqpTOFQicuONNxgRaALhaZ9UA1s6k6keMQNTOU0jpK...,1/24/21 having issues with the app,0.000000
30,1/26/21,gp:AOqpTOEVlTrH8M0CTZC4Ce-14LLz3wK6IBrYJV84tnF...,5 stars for this cool up i really like ot beco...,0.350000
50,1/2/21,gp:AOqpTOGfOzVPTGC1kMiYQ8CvelsxFxhjT37PZFOn51D...,A good application for entertainment,0.700000
59,1/29/21,gp:AOqpTOFb3ce2iH4C1ziXbhXs0BtrtKvZaCiGL-t55Xy...,A little glitchy,-0.187500
66,1/16/21,gp:AOqpTOEVHOgTH3VIR5m-HjcnJoSxM8XxhbQN6uHr8L-...,A very addicting app i love it its fun,0.333333
...,...,...,...,...
3036,9/14/21,gp:AOqpTOHMJvJiILS3s6KprRo3MvYgTgTNXjq6TtuTMet...,Awesame app but Pakistan his ban,0.000000
3037,9/14/21,gp:AOqpTOHc5wHpPiDwwdMPqhgFDDbI7eed5yXpzrGHr_M...,Awesom love it,0.500000
3040,9/11/21,gp:AOqpTOHZTxbM_ZZ1C_esfhT6lGfwhyIIrXdThS5bffL...,Awesome App i like u,1.000000
3048,9/4/21,gp:AOqpTOG-v_HOHpC4JzycMOiye-y9dDbWjJaD4q6ZkHJ...,Awesome apps bro,1.000000


In [40]:
# NLP Text2Emotion python function
def emotion_detection_text2emotion(x):
    all_emotions_value = te.get_emotion(x)
    Keymax_value = max(zip(all_emotions_value.values(), all_emotions_value.keys()))[1]
    return Keymax_value

In [41]:
# applies the Text2Emotion function (see above) and create a new emotion status column
result['emotion_text2emotion'] = result['Review Text'].apply(emotion_detection_text2emotion).astype(str)
result

,ï»¿Posted Date,Review Id,Review Text,polarity,emotion_text2emotion
14,1/25/21,gp:AOqpTOFQicuONNxgRaALhaZ9UA1s6k6keMQNTOU0jpK...,1/24/21 having issues with the app,0.000000,Surprise
30,1/26/21,gp:AOqpTOEVlTrH8M0CTZC4Ce-14LLz3wK6IBrYJV84tnF...,5 stars for this cool up i really like ot beco...,0.350000,Surprise
50,1/2/21,gp:AOqpTOGfOzVPTGC1kMiYQ8CvelsxFxhjT37PZFOn51D...,A good application for entertainment,0.700000,Happy
59,1/29/21,gp:AOqpTOFb3ce2iH4C1ziXbhXs0BtrtKvZaCiGL-t55Xy...,A little glitchy,-0.187500,Surprise
66,1/16/21,gp:AOqpTOEVHOgTH3VIR5m-HjcnJoSxM8XxhbQN6uHr8L-...,A very addicting app i love it its fun,0.333333,Happy
...,...,...,...,...,...
3036,9/14/21,gp:AOqpTOHMJvJiILS3s6KprRo3MvYgTgTNXjq6TtuTMet...,Awesame app but Pakistan his ban,0.000000,Sad
3037,9/14/21,gp:AOqpTOHc5wHpPiDwwdMPqhgFDDbI7eed5yXpzrGHr_M...,Awesom love it,0.500000,Happy
3040,9/11/21,gp:AOqpTOHZTxbM_ZZ1C_esfhT6lGfwhyIIrXdThS5bffL...,Awesome App i like u,1.000000,Surprise
3048,9/4/21,gp:AOqpTOG-v_HOHpC4JzycMOiye-y9dDbWjJaD4q6ZkHJ...,Awesome apps bro,1.000000,Surprise


In [43]:
# Export final dataframe
result.to_csv("Your folder path goes here")